## Bartosz Kosieradzki
#### 2023-01-07

## Introduction

This report provides a detailed analysis of the solutions to assignment number 2 in course Data Processing in R and Python. The report includes description of the solutions and comparison of correctness. Each query from the assignment is discussed in detail in each subsection in the report. For comparing results there is used .equals() function.

At first I there are imported csv files and the conntection with database is made to read sql queries.

In [66]:
import numpy as np
import pandas as pd
import os.path
import sqlite3
import tempfile

from Kosieradzki_Bartosz_assignment_2 import *

Posts = pd.read_csv(r"Posts.csv")
Badges = pd.read_csv(r"Badges.csv")
Comments = pd.read_csv(r"Comments.csv")
Users = pd.read_csv(r"Users.csv")
Votes = pd.read_csv(r"Votes.csv")

baza = os.path.join(tempfile.mkdtemp(), 'example.db')
if os.path.isfile(baza):
    os.remove(baza)

conn = sqlite3.connect(baza)

Badges.to_sql("Badges", conn)
Comments.to_sql("Comments", conn)
Posts.to_sql("Posts", conn)
Users.to_sql("Users", conn)
Votes.to_sql("Votes", conn)

res1 = pd.read_sql_query("""
                        SELECT STRFTIME('%Y', CreationDate) AS Year, COUNT(*) AS TotalNumber
                           FROM Posts
                           GROUP BY Year
                        """, conn)

res2 = pd.read_sql_query("""
                                    SELECT Id, DisplayName, SUM(ViewCount) AS TotalViews
                                    FROM Users
                                    JOIN (
                                    SELECT OwnerUserId, ViewCount FROM Posts WHERE PostTypeId = 1
                                    ) AS Questions
                                    ON Users.Id = Questions.OwnerUserId
                                    GROUP BY Id
                                    ORDER BY TotalViews DESC
                                    LIMIT 10
                                    """, conn)

res3 = pd.read_sql_query("""
                                    SELECT Year, Name, MAX((Count * 1.0) / CountTotal) AS MaxPercentage
                                    FROM (
                                    SELECT BadgesNames.Year, BadgesNames.Name, BadgesNames.Count, BadgesYearly.CountTotal
                                    FROM (
                                    SELECT Name, COUNT(*) AS Count, STRFTIME('%Y', Badges.Date) AS Year
                                    FROM Badges
                                    GROUP BY Name, Year
                                    ) AS BadgesNames
                                    JOIN (
                                    SELECT COUNT(*) AS CountTotal, STRFTIME('%Y', Badges.Date) AS Year
                                    FROM Badges
                                    GROUP BY YEAR
                                    ) AS BadgesYearly
                                    ON BadgesNames.Year = BadgesYearly.Year
                                    )
                                    GROUP BY Year
                                    """, conn)

res4 = pd.read_sql_query("""
                                    SELECT Title, CommentCount, ViewCount, CommentsTotalScore, DisplayName, Reputation, Location
                                    FROM (
                                    SELECT Posts.OwnerUserId, Posts.Title, Posts.CommentCount, Posts.ViewCount,
                                    CmtTotScr.CommentsTotalScore
                                    FROM (
                                    SELECT PostId, SUM(Score) AS CommentsTotalScore
                                    FROM Comments
                                    GROUP BY PostId
                                    ) AS CmtTotScr
                                    JOIN Posts ON Posts.Id = CmtTotScr.PostId
                                    WHERE Posts.PostTypeId=1
                                    ) AS PostsBestComments
                                    JOIN Users ON PostsBestComments.OwnerUserId = Users.Id
                                    ORDER BY CommentsTotalScore DESC LIMIT 10
                                    """, conn)

res5 = pd.read_sql_query("""
                                        SELECT Posts.Title, STRFTIME('%Y-%m-%d', Posts.CreationDate) AS Date, VotesByAge.*
                                        FROM Posts
                                        JOIN (
                                        SELECT PostId,
                                        MAX(CASE WHEN VoteDate = 'before' THEN Total ELSE 0 END) BeforeCOVIDVotes,
                                        MAX(CASE WHEN VoteDate = 'during' THEN Total ELSE 0 END) DuringCOVIDVotes,
                                        MAX(CASE WHEN VoteDate = 'after' THEN Total ELSE 0 END) AfterCOVIDVotes,
                                        SUM(Total) AS Votes
                                        FROM (
                                        SELECT PostId,
                                        CASE STRFTIME('%Y', CreationDate)
                                        WHEN '2022' THEN 'after'
                                        WHEN '2021' THEN 'during'
                                        WHEN '2020' THEN 'during'
                                        WHEN '2019' THEN 'during'
                                        ELSE 'before'
                                        END VoteDate, COUNT(*) AS Total
                                        FROM Votes
                                        WHERE VoteTypeId IN (3, 4, 12)
                                        GROUP BY PostId, VoteDate
                                        ) AS VotesDates
                                        GROUP BY VotesDates.PostId
                                        ) AS VotesByAge ON Posts.Id = VotesByAge.PostId
                                        WHERE Title NOT IN ('') AND DuringCOVIDVotes > 0
                                        ORDER BY DuringCOVIDVotes DESC, Votes DESC
                                        LIMIT 20
                                        """, conn)


conn.close()

### Query 1

The first query in the assignment asks for the number of posts made in each year. The resulting output should be a list of years and the corresponding number of posts made in each year. 

#### Equivalence test

In [67]:
solution_1(Posts).equals(res1)

True

### Query 2

The second query returns the id and name of the top 10 authors of posts from table Users who have the most views on their posts.

#### Equivalence test

In [68]:
solution_2(Users, Posts).equals(res2)

True

### Query 3

The third query for each year returns the most commonly awarded badge and how frequent it was.

#### Equivalence test

In [69]:
solution_3(Badges).equals(res3)

True

### Query 4

The fourth query provides a list of the top 10 posts with the highest cumulative comments score, along with detailed information for each post, including the title, the total number of comments, the number of views, the name of the author, location and their reputation. 

#### Equivalence test

In [70]:
solution_4(Comments, Posts, Users).equals(res4)

True

### Query 5

The fifth query calculates the number of votes each post received before, during, and after COVID-19. It filters out posts that did not receive any votes during COVID-19, and orders the results by the number of during-COVID votes and total votes, showing the top 20 posts, along with its title, date, id, number of votes before COVID, during COVID, after COVID and the total number of votes.

#### Equivalence test

In [71]:
solution_5(Posts, Votes).equals(res5)

True

### Summary

In summary, all solutions from sql matched their equivalent solutions in pandas, which was sometimes difficult because of non-matching data types. 